In [ ]:
from operator import rshift
import cv2 as cv
import cv2
import numpy as np
import mediapipe as mp 
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

mp_drawing  = mp.solutions.drawing_utils

# old

In [ ]:
# left eyes landmarks
LEFT_EYE  = [ 362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398 ]
# right eyes landmarks
RIGHT_EYE = [ 33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246 ] 

LEFT_IRIS  = [474,475, 476, 477]
RIGHT_IRIS = [469, 470, 471, 472]

Left_eb = [107,66,105,63,70,46,53,52,65,55]


######################
#'.\\Data\\Train_video.mp4' r'.\previous-MLcode\Train_video.mp4'
cap = cv.VideoCapture(0)
#DIMS  = np.array([460, 880], dtype=int)      #Video crop dimension to subject
#SCALE = 0.75
######################

with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
) as face_mesh:
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame = cv.flip(frame, 1)
        rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        img_h, img_w = frame.shape[:2]
        results = face_mesh.process(rgb_frame)
        
        ############################################
        '''
        frame = frame[300:300+DIMS[1], 80:150+DIMS[0]]
        frame = cv.resize(frame, (0, 0), fx = SCALE, fy = SCALE)
        frame = cv.flip(frame,0)
        img_h, img_w = frame.shape[:2]
        results = face_mesh.process(frame)
        '''
        ############################################
        '''
        for face_landmark in results.multi_face_landmarks: 
        
            mp_drawing.draw_landmarks(
                    image = frame,
                    landmark_list = mesh_points,
                    connections   = mp_face_mesh.FACEMESH_TESSELATION,
        
                    landmark_drawing_spec = mp_drawing.DrawingSpec(
                        color=(255,0,0),
                        thickness=0,
                        circle_radius=1),
                    
                    connection_drawing_spec = mp_drawing.DrawingSpec(
                        color=(255,255,255),
                        thickness=1,
                        circle_radius=1)    )
        '''
        if results.multi_face_landmarks:
            #print(results.multi_face_landmarks[0].landmark)
            
            mesh_points = np.array([np.multiply([p.x, p.y], [img_w, img_h]).astype(int) for p in results.multi_face_landmarks[0].landmark])
            # multiplying with height and width to get the actual coordinates of the landmarks from {x:0.56 y:0.21} to 200, 300 etc.
            
            # print(mesh_points.shape)
            
            
            cv.polylines(frame, [mesh_points[LEFT_IRIS]], True, (0,255,0), 1, cv.LINE_AA)
            cv.polylines(frame, [mesh_points[RIGHT_IRIS]], True, (0,255,0), 1, cv.LINE_AA)
            
            #print(results.multi_face_landmarks[0].landmark)
        cv.imshow('img', frame)
        key = cv.waitKey(1)
        if key ==ord('q'):
            break
cap.release()
cv.destroyAllWindows()

In [ ]:
a = results.multi_face_landmarks


# XYZ

In [11]:
# https://mediapipe.readthedocs.io/en/latest/solutions/face_detection.html

class FaceDetector:
    '''
    FaceDetector is used to get the 'bounds' for a face.
    'bounds' are used to crop the image befor eseonding it for face/iris landmark detection.
    '''
    def __init__(self):
        # creating detector object
        with open(r".\data\blaze_face_short_range.tflite", "rb") as model_file:
            model_data = model_file.read()
        
        options = vision.FaceDetectorOptions(
            base_options = python.BaseOptions(model_asset_buffer=model_data),
            running_mode = vision.RunningMode.IMAGE
        )
        
        self.face_detector = vision.FaceDetector.create_from_options(options)
        
    def detect_face_bounds(self, image:np.array) -> tuple:
        # convert nump image to mediapipe format
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
        
        # detect face in image
        self.face_detection_result = self.face_detector.detect(mp_image)
    
        # if face detected, draw on image and return bounds
        if self.face_detection_result.detections:
            bbox = self.face_detection_result.detections[0].bounding_box
            self.draw_bounds(image, bbox)
            return self.expand_bounds(bbox, scale=2)      # double width and height

        print("No face detected")
        return None

    
    def draw_bounds(self, image:np.array, bbox) -> tuple:
        height, width, _ = image.shape
        start_point = bbox.origin_x, bbox.origin_y
        end_point   = bbox.origin_x + bbox.width, bbox.origin_y + bbox.height
        
        cv2.rectangle(image, start_point, end_point, (255,0,0), 3)

    
    def expand_bounds(self, bbox, scale:int=2):
        x = max(0, round( bbox.origin_x - bbox.width*(scale-1)/2 )  )
        y = max(0, round( bbox.origin_y - bbox.height*(scale-1)/2 ) )

        print(x,y)
        bbox.x = x
        bbox.y = y
        bbox.width  = round(scale*bbox.width)
        bbox.height = round(scale*bbox.height)

        return bbox

In [15]:
class FaceMeshDetector:
    def __init__(self):
        self.LEFT_IRIS  = [474,475, 476, 477]
        self.RIGHT_IRIS = [469, 470, 471, 472]
        self.mp_face_mesh = mp.solutions.face_mesh
        self.face_mesh    = self.mp_face_mesh.FaceMesh(refine_landmarks=True)

    def detect(self, image):
        results = self.face_mesh.process(image)
        self.draw_face_landmarks(image, results)   # OPTIONAL
        
        return results
    
    def draw_face_landmarks(self, image, results):
        if not results.multi_face_landmarks:
            return
        #iris
        img_h, img_w = image.shape[:2]        
        mesh_points = np.array([np.multiply([p.x, p.y], [img_w, img_h]).astype(int) for p in results.multi_face_landmarks[0].landmark])
        # multiplying with height and width to get the actual coordinates of the landmarks from {x:0.56 y:0.21} to 200, 300 etc.
        cv.polylines(image, [mesh_points[self.LEFT_IRIS]], True, (0,255,0), 1, cv.LINE_AA)
        cv.polylines(image, [mesh_points[self.RIGHT_IRIS]], True, (0,255,0), 1, cv.LINE_AA)
        return
        
        #facial landmarks
        for face_landmark in results.multi_face_landmarks:
            mp_drawing.draw_landmarks(
                image = image,
                landmark_list = face_landmark,
                connections   = self.mp_face_mesh.FACEMESH_TESSELATION,
    
                landmark_drawing_spec = mp_drawing.DrawingSpec(
                    color=(255,0,0),
                    thickness=0,
                    circle_radius=1),
                
                connection_drawing_spec = mp_drawing.DrawingSpec(
                    color=(255,255,255),
                    thickness=1,
                    circle_radius=1)    )

In [16]:
face_bounds_detect = FaceDetector()
face_mesh_detector = FaceMeshDetector()

In [17]:
# cam = cv2.VideoCapture(0)
cam = cv2.VideoCapture(r".\previous-MLcode\Train_video.mp4")

# get face bounds in image
bbox = None
while bbox is None:
    _, frame = cam.read()
    bbox = face_bounds_detect.detect_face_bounds(frame)  # keep this before while True


# process video
#####

####
while True:
    _, frame = cam.read()
    if not _: break

    cropx256 = cv2.resize(frame[bbox.y : bbox.y+bbox.width, bbox.x : bbox.x+bbox.height], (256,256))
    #put results = iris.detect(cropx256)
    results = face_mesh_detector.detect(cropx256)

    cv2.imshow("cam", cropx256)
        
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()

228 288
